In [1]:
!pip install pytesseract pymupdf


In [2]:
import pytesseract

# Ví dụ Windows: đường dẫn tới tesseract.exe
pytesseract.pytesseract.tesseract_cmd = r"C:\Users\likgn\AppData\Local\Programs\Tesseract-OCR\tesseract.exe"


In [4]:
import pymupdf  # PyMuPDF
import pytesseract
from PIL import Image
import io

# Mở file PDF
pdf_path = r"C:\Users\likgn\Repository\RAG\documents\one_page_latex_document.pdf"
doc = pymupdf.open(pdf_path)

text = ""
for page_num in range(len(doc)):
    page = doc[page_num]
    # Xuất trang dưới dạng ảnh (300dpi)
    pix = page.get_pixmap(dpi=300)
    img = Image.open(io.BytesIO(pix.tobytes("png")))
    
    # OCR với tiếng Việt: lang="vie"
    page_text = pytesseract.image_to_string(img, lang="equ")
    text += f"\n--- Trang {page_num+1} ---\n{page_text}"

# Lưu kết quả
with open("output.txt", "w", encoding="utf-8") as f:
    f.write(text)

print("Hoàn tất OCR và lưu vào output.txt")


TesseractError: (1, 'Error opening data file C:\\Users\\likgn\\AppData\\Local\\Programs\\Tesseract-OCR/tessdata/equ.traineddata Please make sure the TESSDATA_PREFIX environment variable is set to your "tessdata" directory. Failed loading language \'equ\' Tesseract couldn\'t load any languages! Could not initialize tesseract.')

In [5]:
from enum import Enum
import os
from pathlib import Path
from typing import List, Iterator
from pydantic import BaseModel, Field
from langchain_core.documents import Document
from langchain_community.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter
from langchain_nvidia_ai_endpoints import ChatNVIDIA
from langchain.prompts import ChatPromptTemplate

class ChunkingStrategy(str, Enum):
    """Enum định nghĩa các chiến lược chunking khác nhau."""
    ONE_PAGE_PER_CHUNK = "one_page_per_chunk"  # Mỗi trang là một chunk
    RECURSIVE_SPLIT = "recursive_character_split"  # Sử dụng RecursiveCharacterTextSplitter
    LLM_SPLIT = "llm_split"

class ThematicBlock(BaseModel):
    """Represents a single thematic block in the document"""
    summary_title: str = Field(
        description="The title of the thematic group. Don't add index numbers."
    )
    content: str = Field(
        description="The extracted content of the thematic group."
    )
    start_page_index: int = Field(
        default=0,
        description="The starting page index of the thematic group."
    )
    end_page_index: int = Field(
        default=0,
        description="The ending page index of the thematic group."
    )

class ThematicGroupList(BaseModel):
    """A list of thematic blocks, each representing an extracted thematic block."""
    thematic_group_list: List[ThematicBlock] = Field(
        default_factory=list,
        description="List of thematic blocks extracted from the document"
    )


In [9]:
llm = ChatNVIDIA(
                    model="openai/gpt-oss-20b",
                    api_key="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",
                    temperature=0.1,
                    max_completion_tokens=15000
                )
llm.invoke("hello")

AIMessage(content='Hello! 👋 How can I help you today?', additional_kwargs={'reasoning_content': 'The user says "hello". We need to respond. The user hasn\'t asked a question. We can respond with a friendly greeting. The user might want to start a conversation. We can ask how we can help. The user might want to talk about something. We can respond with a friendly greeting and ask what they need. There\'s no special instructions. So we can respond with a friendly greeting.'}, response_metadata={'role': 'assistant', 'reasoning_content': 'The user says "hello". We need to respond. The user hasn\'t asked a question. We can respond with a friendly greeting. The user might want to start a conversation. We can ask how we can help. The user might want to talk about something. We can respond with a friendly greeting and ask what they need. There\'s no special instructions. So we can respond with a friendly greeting.', 'content': 'Hello! 👋 How can I help you today?', 'refusal': None, 'annotations

In [ ]:
self.prompt_template = ChatPromptTemplate.from_messages([
                ("system", """Reasoning:low. 
                            Your role includes handling text splitting tasks for the RAG pipeline. 
                            Perfect chunking isn’t about blindly cutting at every paragraph break — it’s about finding 
                            the sweet spot between semantic coherence and retrieval efficiency.""" ),
                ("user", self._get_llm_text_splitter_prompt() + 
                        """\n------------BEGIN-----------\n
                        {text_to_split}\n
                        -----------END----------------""")
            ])